In [1]:
## Env

import nibabel as nib
from nilearn import datasets, input_data, plotting, connectome, image
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cdist
from scipy.stats import pearsonr, zscore
import pandas as pd
import os
import json
import glob
import gc
from joblib import Parallel, delayed
from tqdm import tqdm


### Testes

In [2]:
df_extended = []

for sub_id in range(1, 87):
    try:
        # read
        path =  glob.glob(f"../data/fmri_derived/dfcs/v1/sub-{sub_id}_*.parquet")
        data = pd.read_parquet(path)
        data = data.drop(columns=['sub','movie','step'])
        data = data.set_index(['start', 'node'])
        data = data.unstack(level=-1)
        
        # identifica os indices 
        dummy = np.arange(14884).reshape(122, 122)
        clean_indices = np.triu_indices_from(dummy, k=1)
        flattened_clean = dummy[clean_indices]
        
        # dropa as colunas desses indices
        data.columns = np.arange(14884)
        data = data[flattened_clean]

        # add sub
        data['sub'] = sub_id
        data = data.set_index('sub', append=True)

        # salva            
        if sub_id == 1:
            df_extended = data
        else:
            df_extended = pd.concat([df_extended, data])

    except Exception as e:
        print(f"Ocorreu um erro sub {sub_id}: {e}")

# Clean Memory
del data, dummy, clean_indices, flattened_clean
gc.collect() 

# Show
df_extended      

Ocorreu um erro sub 49: "['sub', 'movie', 'step'] not found in axis"


,,1,2,3,4,5,6,7,8,9,10,...,14392,14393,14394,14395,14515,14516,14517,14638,14639,14761
start,sub,,,,,,,,,,,,,,,,,,,,,
0,1,0.060413,0.196270,0.289575,0.475760,0.183561,0.191599,0.456325,0.313982,0.313079,0.227664,...,0.366860,0.297321,0.066651,0.619848,0.391654,0.393771,0.408156,0.568355,0.495100,0.291028
75,1,0.119580,0.165901,0.253316,0.416897,0.189053,0.115492,0.437765,0.369510,0.317792,0.226243,...,0.384259,0.295853,0.104757,0.557777,0.326469,0.314284,0.347007,0.569302,0.469473,0.314702
225,1,0.155231,0.209647,0.199147,0.303766,0.189499,0.037618,0.237875,0.356512,0.320383,0.235249,...,0.530087,0.332032,0.255028,0.438802,0.301895,0.311412,0.300272,0.538806,0.486981,0.356773
300,1,0.136793,0.163141,0.084268,0.376580,0.167698,0.060675,0.202105,0.395682,0.365891,0.214626,...,0.388894,0.341538,0.229864,0.493160,0.338118,0.316897,0.303625,0.470696,0.440078,0.270994
375,1,0.162481,0.161755,0.022194,0.456571,0.252898,0.069465,0.113957,0.409092,0.357889,0.203600,...,0.436909,0.414124,0.306680,0.607459,0.442443,0.463079,0.415608,0.519637,0.546090,0.347744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7050,86,0.076031,0.181170,0.289520,0.263805,0.173701,0.323789,0.490290,0.116312,0.201178,0.125548,...,0.479235,0.432278,0.231849,0.404289,0.451054,0.279699,0.404944,0.238448,0.517485,0.123185
7125,86,0.056964,0.196296,0.253865,0.224134,0.161490,0.263112,0.383797,0.012632,0.221759,0.125840,...,0.485017,0.468451,0.194492,0.395709,0.542478,0.245999,0.377404,0.194743,0.544031,0.098245
7200,86,0.058448,0.127680,0.199761,0.188692,0.082445,0.260676,0.361389,0.013718,0.221226,0.094060,...,0.323067,0.367335,0.146824,0.203412,0.378413,0.176871,0.269938,0.146589,0.449235,0.142265


In [3]:
df_extended = df_extended.reset_index()
# save data
df_extended.drop(columns=["start", "sub"]).to_parquet("../data/fmri_derived/dfcs/all_dfcs_flattened.parquet")
# save index
df_extended[["start", "sub"]].to_csv("../data/fmri_derived/dfcs/all_dfcs_flattened_index.csv")

In [4]:
df = pd.read_parquet("../data/fmri_derived/dfcs/all_dfcs_flattened.parquet")
df

MemoryError: 

In [ ]:
df_sub = df_extended.drop(columns="start").groupby('sub').mean()
df_sub.to_csv("../data/fmri_derived/dfcs/all_dfcs_flattened_sub.parquet") # Nao é FC, mas é algo, tira a media nao é igual a fazer a correlacao de tudo, mas nem acho q vou usar isso
df_sub